# Attemp number 2
This attemp includes findings from my previous attemp, bringing them together to form a more refined approach at producing a fully cleaned dataset

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

script_path = Path.cwd()
data_path_cleaned = script_path.parent / "data" / "games_cleaned.csv"

df = pd.read_csv(data_path_cleaned, encoding='utf-8')

# Dtatype conversion
df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
print(df.dtypes)

AppID                                int64
Name                                object
Release date                datetime64[ns]
Estimated owners                    object
Required age                         int64
Price                              float64
User score                           int64
Positive                             int64
Negative                             int64
Recommendations                      int64
Average playtime forever             int64
Developers                          object
Publishers                          object
Categories                          object
Genres                              object
Tags                                object
dtype: object


## Estimated average owners

In [2]:
# Split 'Estimated owners' into two columns and calculate the average
owners_clean = df['Estimated owners'].str.replace(' ', '', regex=False)

owners_split = owners_clean.str.split('-', expand=True)

owners_split[0] = pd.to_numeric(owners_split[0], errors='coerce')
owners_split[1] = pd.to_numeric(owners_split[1], errors='coerce')

df['Estimated average owners'] = (owners_split[0] + owners_split[1]) / 2
df['Estimated average owners'] = df['Estimated average owners'].astype('Int64')

print(df[['Estimated owners', 'Estimated average owners']].head(10))

  Estimated owners  Estimated average owners
0        0 - 20000                     10000
1        0 - 20000                     10000
2        0 - 20000                     10000
3        0 - 20000                     10000
4        0 - 20000                     10000
5   50000 - 100000                     75000
6        0 - 20000                     10000
7        0 - 20000                     10000
8        0 - 20000                     10000
9   50000 - 100000                     75000


## Reranging the dataset

In [15]:
# Adding new column for lowercase names
df['Name_lowercase'] = df['Name'].str.lower().str.strip()

# Check for duplicates
duplicates = df[df['Name_lowercase'].duplicated(keep=False)]
print(duplicates.shape)

(2655, 27)


### Positive ratio
As we move beyond simple ratios, using only the share of positive reviews can be misleading since it ignores how many reviews support that score. A game with 9 out of 10 positives looks the same as one with 9,000 out of 10,000, even though the latter is far more reliable.

The `Bayesian average` fixes this by combining each game’s observed ratio with a prior mean, weighted by how much evidence we have. With few reviews, the score stays near the prior; as reviews grow, it converges to the game’s own ratio. This gives a stable and interpretable quality estimate.

Let $\bar{x}$ be the game’s observed positive ratio, $n$ the number of reviews, $\mu$ the prior mean (e.g., the global average across all games), and $m$ the prior strength (how many “virtual reviews” the prior contributes). The Bayesian score is:
$$
\text{Bayesian Score} \;=\; \frac{\bar{x}\,n + \mu\,m}{n + m}
$$
Equivalently, with a Beta prior $\mathrm{Beta}(\alpha,\beta)$ where $\alpha=m\mu$ and $\beta=m(1-\mu)$ and observed positives $x$ out of $n$:
$$
\text{Bayesian Score} \;=\; \frac{x+\alpha}{n+\alpha+\beta}
$$
**Notes**
- Choose $\mu$ as the global positive rate, or a fixed value like $0.7$ if you want a stable baseline.
- Choose $m$ in the range 50–200 for Steam-like data. Larger $m$ pulls small-sample games more strongly toward $\mu$.
- Keep the raw ratio separately for display; use the Bayesian score for ranking.

In [ ]:
# Bayesian average
df["n_reviews"] = df["Positive"] + df["Negative"]
df["positive ratio"] = df["Positive"] / df["n_reviews"].replace(0, np.nan)

print("Mean:", df["n_reviews"].mean())
print("Median:", df["n_reviews"].median())
print("Min:", df["n_reviews"].min())
print("Max:", df["n_reviews"].max())
print("Quantiles:", df["n_reviews"].quantile([0.25,0.5,0.75,0.9,0.99]))


(2655, 27)
Mean: 880.2117054875641
Median: 5.0
Min: 0
Max: 6531097
Quantiles: 0.25        0.0
0.50        5.0
0.75       39.0
0.90      293.0
0.99    11618.0
Name: n_reviews, dtype: float64


In [ ]:

# Global mean for prior
mu = df["Positive"].sum() / df["n_reviews"].sum()

def bayesian_score(pos_ratio, n, mu, m):
    return ((pos_ratio * n + mu * m) / (n + m)).fillna(mu)

# Compute for different m values
for m in [50,100,150,200]:
    df[f"bayes_m{m}"] = bayesian_score(df["positive_ratio"], df["n_reviews"], mu, m)

# Filter small samples
df_filtered = df[df["n_reviews"] >= 20].copy()

# Show top 10 for each m
for m in [50,100,150,200]:
    print(f"\nTop 10 games by Bayesian average (m={m})")
    print(
        df_filtered.sort_values("n_reviews", ascending=False)
        [["n_reviews","Positive", "Negative", "positive_ratio", f"bayes_m{m}"]]
        .head(10)
    )

In [ ]:

df_sorted = df.sort_values(
    by=['Estimated average owners', 'Positive ratio', 'Release date'],
    ascending=[False, False, False]
)

print(df_sorted[['Estimated average owners','Positive ratio','Release date']].head(10))

### Missing values

In [4]:
# Initial exploration of missing values
print(df.isnull().sum())

AppID                           0
Name                            6
Release date                  131
Estimated owners                0
Required age                    0
Price                           0
User score                      0
Positive                        0
Negative                        0
Recommendations                 0
Average playtime forever        0
Developers                   6475
Publishers                   6778
Categories                   7566
Genres                       6440
Tags                        37423
Estimated average owners        0
Name_lowercase                  6
Positive ratio              37588
dtype: int64


In [5]:
# Number of missing values in major columns that can be found in duplicates
columns_to_check = ['Developers', 'Publishers', 'Categories', 'Genres', 'Tags']

for col in columns_to_check:
    missing_rows = df[df[col].isnull()]
    
    can_be_filled = missing_rows['Name_lowercase'].isin(
        df[~df[col].isnull()]['Name_lowercase']
    )
    
    fillable_rows = missing_rows[can_be_filled]
    
    print(f"{col}: {len(fillable_rows)} rows can be filled from duplicates.")


Developers: 16 rows can be filled from duplicates.
Publishers: 26 rows can be filled from duplicates.
Categories: 31 rows can be filled from duplicates.
Genres: 13 rows can be filled from duplicates.
Tags: 595 rows can be filled from duplicates.



To fill the datas across duplicated game entries, I grouped rows by `Name_lowercase` and updated using `.update` with `.ffill` and `.bfill` to propagate only missing values within each group. This way, no matter which duplicate held the information, all rows for the same game now share the filled values


In [6]:
# Filling missing values from the duplicates
cols_to_fill = ['Developers', 'Publishers', 'Categories', 'Genres', 'Tags']

for col in cols_to_fill:
    df.update(df.groupby('Name_lowercase')[col].ffill())
    df.update(df.groupby('Name_lowercase')[col].bfill())


In [7]:
# Double checking the missing values after filling
cols_to_check = ['Developers', 'Publishers', 'Categories', 'Genres', 'Tags']

for col in cols_to_check:
    missing_rows = df[df[col].isnull()]
    
    can_be_filled = missing_rows['Name_lowercase'].isin(
        df[~df[col].isnull()]['Name_lowercase']
    )
    
    fillable_rows = missing_rows[can_be_filled]
    
    print(f"{col}: {len(fillable_rows)} rows can be filled from duplicates.")


Developers: 0 rows can be filled from duplicates.
Publishers: 0 rows can be filled from duplicates.
Categories: 3 rows can be filled from duplicates.
Genres: 2 rows can be filled from duplicates.
Tags: 0 rows can be filled from duplicates.


After filling missing data within duplicate groups, 5 rows are still empty, althought is being detected as the duplicate contains the much needed information, but my .update approach does not seem to work for this specific situations.

This may be because of 1. small differences in how the groups were formed or 2. because the update step did not replace the missing values.  
Since the number is small, the next step is to check these rows directly or combine the duplicates into a single row per game.

In [8]:
print(df.isnull().sum())

AppID                           0
Name                            6
Release date                  131
Estimated owners                0
Required age                    0
Price                           0
User score                      0
Positive                        0
Negative                        0
Recommendations                 0
Average playtime forever        0
Developers                   6459
Publishers                   6752
Categories                   7538
Genres                       6429
Tags                        36828
Estimated average owners        0
Name_lowercase                  6
Positive ratio              37588
dtype: int64


In [9]:
# Only 6 rows still have missing values, therefore dropping them might be the better option. 
# By dropping these rows, we should also be dropping the rows with missing values in Name_lowercase
df = df.dropna(subset=['Name'])

#filling remaining string columns with 'Unknown'
df['Release date'] = df['Release date'].fillna('Unknown')
df['Developers'] = df['Developers'].fillna('Unknown')
df['Publishers'] = df['Publishers'].fillna('Unknown')
df['Categories'] = df['Categories'].fillna('Unknown')
df['Genres'] = df['Genres'].fillna('Unknown')
df['Tags'] = df['Tags'].fillna('Unknown')

# Filling remaining numeric columns with 0
df['Estimated average owners'] = df['Estimated average owners'].fillna(0)
df['Positive ratio'] = df['Positive ratio'].fillna(0)

print(df.isnull().sum())

AppID                       0
Name                        0
Release date                0
Estimated owners            0
Required age                0
Price                       0
User score                  0
Positive                    0
Negative                    0
Recommendations             0
Average playtime forever    0
Developers                  0
Publishers                  0
Categories                  0
Genres                      0
Tags                        0
Estimated average owners    0
Name_lowercase              0
Positive ratio              0
dtype: int64


### Duplicates

In [10]:
# Sorting by the priorities
df_sorted = df.sort_values(
    by=['Estimated average owners', 'Positive ratio', 'Release date'],
    ascending=[False, False, False]
)

# Keeping the first duplicate as its being sorted already
df_drop = df_sorted.drop_duplicates('Name_lowercase', keep='first')

data_path_final = script_path.parent / "data" / "games_final.csv"
df_drop.to_csv(data_path_final, index=False, encoding="utf-8")

print("cleaned")

cleaned


In [11]:
# Double checking the duplicates in the final cleaned file
df2 = pd.read_csv(data_path_final, encoding='utf-8')

duplicates2 = df2[df2['Name_lowercase'].duplicated(keep=False)]
print(duplicates2.shape)

(0, 19)
